In [18]:
import pickle

In [19]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [20]:
from langchain_community.vectorstores import FAISS

In [21]:
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

In [22]:
with open(f"vectorestore/7QxDbPTpUlJY27ZKC2PY.pkl", "rb") as f:
        VectorStore = pickle.load(f)

In [23]:
query = "Who is the Chief executive officer of this company"

In [25]:
docs=VectorStore.similarity_search(query, k=4)

In [26]:
docs

[Document(page_content='(w.e.f May 31,2021)\nDinesh Kalani, Company Secretary\nGlobal Chief Executive Officer (CEO):\nHiral Chandrana, Global Chief Executive Officer\nDirectors:\nKetan Mehta, Non Executive Director \nAtul Kanagat, Non Executive Director (till Jan \n17,2023)\nPriti Rao, Non Executive Director\nS. Sandilya, Non Executive Director (till March \n3, 2023)\nRajeev Grover, Non Executive Director\nSuresh Vaswani Non Executive Director (w.e.f \nDec 11,2022)\n* Non executive Director and Chairman w.e.f. April 01, 2023\nStandalone Accounts\n215\nStatutory  \nReports\nFinancial\nStatements\nCorporate  \nOverview\nShareholder  \nInformation', metadata={'page': 216}),
 Document(page_content='Company’s strategic initiatives.\n \n‑ \nOversee and evaluate the overall performance of \nthe Board and its Members.\n \n‑ \nEnsure to place all relevant matters before the \nBoard and encourage healthy participation by \nall Directors to enable them to provide their \nexpert guidance.\n \n‑ \n

In [9]:
passages =[]
for doc in docs:
    passages.append({"text":doc.page_content, "metadata":{"page": doc.metadata["page"]}})

In [10]:
passages

[{'text': '(w.e.f May 31,2021)\nDinesh Kalani, Company Secretary\nGlobal Chief Executive Officer (CEO):\nHiral Chandrana, Global Chief Executive Officer\nDirectors:\nKetan Mehta, Non Executive Director \nAtul Kanagat, Non Executive Director (till Jan \n17,2023)\nPriti Rao, Non Executive Director\nS. Sandilya, Non Executive Director (till March \n3, 2023)\nRajeev Grover, Non Executive Director\nSuresh Vaswani Non Executive Director (w.e.f \nDec 11,2022)\n* Non executive Director and Chairman w.e.f. April 01, 2023\nStandalone Accounts\n215\nStatutory  \nReports\nFinancial\nStatements\nCorporate  \nOverview\nShareholder  \nInformation',
  'metadata': {'page': 216}},
 {'text': 'Company’s strategic initiatives.\n \n‑ \nOversee and evaluate the overall performance of \nthe Board and its Members.\n \n‑ \nEnsure to place all relevant matters before the \nBoard and encourage healthy participation by \nall Directors to enable them to provide their \nexpert guidance.\n \n‑ \nMonitor the performan

In [ ]:
from flashrank import Ranker, RerankRequest

# Nano (~4MB), blazing fast model & competitive performance (ranking precision).
ranker = Ranker()

In [109]:
rerankrequest = RerankRequest(query=query, passages=passages)

In [110]:
results = ranker.rerank(rerankrequest)

In [111]:
for res in results:
    print(res["metadata"])

{'page': 113}
{'page': 291}
{'page': 217}
{'page': 119}
{'page': 120}
{'page': 118}
{'page': 129}
{'page': 118}
{'page': 85}
{'page': 58}
{'page': 217}
{'page': 134}
{'page': 113}
{'page': 334}
{'page': 179}
{'page': 54}
{'page': 121}
{'page': 120}
{'page': 153}
{'page': 119}


In [112]:
for doc in docs:
    print(doc.metadata)

{'page': 217}
{'page': 119}
{'page': 85}
{'page': 129}
{'page': 118}
{'page': 217}
{'page': 120}
{'page': 113}
{'page': 291}
{'page': 120}
{'page': 118}
{'page': 334}
{'page': 58}
{'page': 113}
{'page': 134}
{'page': 121}
{'page': 54}
{'page': 153}
{'page': 179}
{'page': 119}


In [18]:
from sentence_transformers import CrossEncoder

reranker_model = CrossEncoder(model_name="bge-reranker-base", max_length=512)

def rerank_docs(query, retrieved_docs):
    query_and_docs = [(query, r.page_content) for r in retrieved_docs]
    scores = reranker_model.predict(query_and_docs)
    return sorted(list(zip(retrieved_docs, scores)), key=lambda x: x[1], reverse=True)

In [19]:
rerank_docs(query, docs)

[(Document(page_content='Directors’ Report   |  72 Integrated Annual Report 2022-23 Directors’ Report To the Members, The Directors present this Integrated Annual Report of   Tata Consultancy Services Limited (the Company or TCS) along  with the audited financial statements for the financial year  ended March 31, 2023. The consolidated performance of the Company and its  subsidiaries has been referred to wherever required. 1.  Financial results (` crore) Standalone Consolidated Financial Year  2022-23 (FY 2023) Financial Year  2021-22 (FY 2022) Financial Year  2022-23 (FY 2023) Financial Year  2021-22 (FY 2022) Revenue from operations 1,90,354 1,60,341 2,25,458 1,91,754 Other income 5,328 7,486 3,449 4,018 Total income 1,95,682 1,67,827 2,28,907 1,95,772 Expenses Operating expenditure 1,39,357 1,14,096 1,66,199', metadata={'page': 72}),
  0.8960131),
 (Document(page_content='as amended from time to time.  Significant accounting policies used in the preparation of the financial statemen

In [8]:
import fitz
import re

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

In [27]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=800,
    chunk_overlap=200,
    length_function=len,
)

In [28]:
doc = fitz.open("Reports/mastek22-23.pdf")
pages=[]
for page_no in range(doc.page_count):
        text = doc[page_no].get_text()
        text = re.sub(r"\n", " ", text)
        text = text_splitter.split_text(text=text)
        for chunk in text:
            page = Document(page_content=chunk, metadata = {"page":page_no+1})
            pages.append(page)  

In [29]:
testStore = FAISS.from_documents(pages, embedding=embeddings)

In [30]:
doc = testStore.similarity_search(query)

In [31]:
doc

[Document(page_content='Chief Executive Officer (CEO): Hiral Chandrana, Global Chief Executive Officer Directors: Ketan Mehta, Non Executive Director  Atul Kanagat, Non Executive Director (till Jan  17,2023) Priti Rao, Non Executive Director S. Sandilya, Non Executive Director (till March  3, 2023) Rajeev Grover, Non Executive Director Suresh Vaswani Non Executive Director (w.e.f  Dec 11,2022) * Non executive Director and Chairman w.e.f. April 01, 2023 Standalone Accounts 215 Statutory   Reports Financial Statements Corporate   Overview Shareholder   Information', metadata={'page': 217}),
 Document(page_content='with  Company’s strategic initiatives.   ‑  Oversee and evaluate the overall performance of  the Board and its Members.   ‑  Ensure to place all relevant matters before the  Board and encourage healthy participation by  all Directors to enable them to provide their  expert guidance.   ‑  Monitor the performance of the Executive  Leadership Team. •  Global Chief Executive Office

In [ ]:
[Document(page_content='(w.e.f May 31,2021)\nDinesh Kalani, Company Secretary\nGlobal Chief Executive Officer (CEO):\nHiral Chandrana, Global Chief Executive Officer\nDirectors:\nKetan Mehta, Non Executive Director \nAtul Kanagat, Non Executive Director (till Jan \n17,2023)\nPriti Rao, Non Executive Director\nS. Sandilya, Non Executive Director (till March \n3, 2023)\nRajeev Grover, Non Executive Director\nSuresh Vaswani Non Executive Director (w.e.f \nDec 11,2022)\n* Non executive Director and Chairman w.e.f. April 01, 2023\nStandalone Accounts\n215\nStatutory  \nReports\nFinancial\nStatements\nCorporate  \nOverview\nShareholder  \nInformation', metadata={'page': 216}),
 Document(page_content='Company’s strategic initiatives.\n \n‑ \nOversee and evaluate the overall performance of \nthe Board and its Members.\n \n‑ \nEnsure to place all relevant matters before the \nBoard and encourage healthy participation by \nall Directors to enable them to provide their \nexpert guidance.\n \n‑ \nMonitor the performance of the Executive \nLeadership Team.\n• \nGlobal Chief Executive Officer is responsible for the \nimplementation of corporate strategy, brand equity \nplanning, external contacts and other management \nmatters which are approved by the Board. He is also \nresponsible for achieving the annual long‑ term business \nplans. His role, inter alia, includes:\n \n‑ \nCrafting of vision and business strategies of \nthe Company.\n \n‑ \nClear understanding and accomplishment of goals \nset by the Board.\n \n‑', metadata={'page': 118}),
 Document(page_content='of the Act read with the Companies (Appointment \nand Remuneration of Managerial Personnel) Rules, \n2014 (as amended from time to time), the following \npersons are acting as the Key Managerial Personnel \n(KMP) of the Company:\n \n \n1. \nMr. Ashank Desai ‑ Chairman & Managing \nDirector (upto March 31, 2023)\n \n \n2. \nMr. Arun Agarwal ‑ Global Chief \nFinancial Officer\n \n \n3. \nMr. Dinesh Kalani ‑ Vice President – Group \nCompany Secretary\n \n \nPursuant to Rule 8(5)(iii) of the Companies \n(Accounts) Rules, 2014, the following change \noccurred in the composition of KMP during the year \nunder review:\n \n \nMr. Ashank Desai relinquished the role of Managing \nDirector of the Company w.e.f. March 31, 2023 from \nclose of business hours, except for this, there was \nno other change in the composition of KMP.', metadata={'page': 84}),
 Document(page_content='required by the Executive Leadership Council of the Group.\nGCEO and GCFO apprises the Board, at each of its Meeting \nabout the overall performance of the Company with \npresentations on business operations and financial affairs \non a regular basis. The Members of Executive Leadership \nCouncil are invited to the Board / Committee Meetings to \nprovide necessary insights into the business performance of \nthe Company and for discussing corporate strategies with the \nBoard / Committee Members.\nThe annual strategic and operating plans of the business are \npresented to the Board. The Quarterly Financial Statements \nand Annual Financial Statements are first presented to \nthe Audit Committee and subsequently to the Board for \ntheir approval. Also, the Risk Management & Governance', metadata={'page': 119}),
 Document(page_content='Global Chief Executive Officer (CEO):\nHiral Chandrana, Global Chief Executive Officer\nDirectors:\nKetan Mehta, Non Executive Director \nAtul Kanagat, Non Executive Director (till January 17,2023)\nPriti Rao, Non Executive Director \nS. Sandilya, Non Executive Director (till March 3, 2023)\nRajeev Grover, Non Executive Director\nSuresh Vaswani Non Executive Director (w.e.f December 11,2022)\nEnterprise where KMP has control:\nMastek Foundation\nNote: For list of subsidiaries refer note 43.\n* Non executive Director and Chairman w.e.f. April 01, 2023\nBalances outstanding are as follows:-\nName of Related Party \n As at\n March 31, 2023\n As at\n March 31, 2022\nCompensation of key management personnel and directors of the Company*\n88\n327', metadata={'page': 290}),
 Document(page_content='of the Board such as Audit Committee, Nomination \nand Remuneration Committee, Corporate Social \nResponsibility Committee, Stakeholders’ Relationship \nCommittee and Risk Management & Governance \nCommittee are focused on financial reporting, audit \nand internal controls, appointment and remuneration \nof Directors, Key Managerial Personnel and Senior \nManagerial Personnel, identifying, implementing \nand monitoring of ESG and CSR activities, the Risk \nmanagement, operational, legal and compliances \nincluding sustainability framework.\n2. \nExecutive Management – by the Corporate Management \nteam comprising of the Managing Director, Chief \nExecutive Officer and Executive Leadership consisting \nof the functional heads of the Company. They meet at', metadata={'page': 112}),
 Document(page_content='May 1, 2023.\nExecutive Leadership Council Members also attend this \nCommittee meeting as permanent invitees and Mr. Dinesh \nKalani – Vice President – Group Company Secretary acts as \nSecretary to the Committee.\nCorporate Governance Report\n127\nShareholder  \nInformation\nStatutory  \nReports\nFinancial\nStatements\nCorporate  \nOverview', metadata={'page': 128}),
 Document(page_content='The Company adheres to the provisions of the Act read with \nthe Rules issued thereunder, Secretarial Standards and the \nSEBI Listing Regulations with respect to convening and holding \nthe Meetings of the Board of Directors, its Committees and \nthe General Meetings of the Members of the Company.\nInvitees and Proceedings\nApart from Board Members and the Vice President – Group \nCompany Secretary, the Board and Committee Meetings \nare generally also attended by the Global Chief Executive \nOfficer (GCEO), Global Chief Financial Officer (GCFO), Global \nChief Legal and Compliance Officer (GC-LCO) and wherever \nrequired by the Executive Leadership Council of the Group.\nGCEO and GCFO apprises the Board, at each of its Meeting \nabout the overall performance of the Company with', metadata={'page': 119}),
 Document(page_content='All Directors are familiarised at regular intervals on various \ntopics specifically relating to: \na) \nnature of the industry in which the company operates;\nb) \nthe business model of the company;\nc) \nabout their roles, rights, and responsibilities in \nthe company; \nd) \nany other relevant information, through various \ninitiatives; and\ne) \nRegulatory Framework within which the Company and \nits other subsidiaries operate.\nEvery new Director of the Board needs to attend a Review \n/ Orientation Program organised by the Company. Global \nChief Executive Officer, Global Chief Financial Officer, and \nSenior Managerial Personnel & Leadership Team, provides \nan overview of the Strategy, Operations and Functions of \nthe Company by making presentations. An opportunity is', metadata={'page': 117}),
 Document(page_content='Management Team\nDelivering Excellence \nwith Expertise \nHiral Chandrana \nChief Executive Officer,  \nMastek Group \nArun Agarwal\nChief Financial Officer, \nMastek Group\nAbhishek Singh\nPresident UKI & Europe\nPrajakta Talvelkar\nChief Marketing Officer\nPrameela Kalive\nChief Operating Officer\n56\nMastek Limited\nAnnual Report 2022-23', metadata={'page': 57}),
 Document(page_content='Step-down Subsidiary\nUnited States of America\nEvosys Consultancy Services (Malaysia) Sdn Bhd \nStep-down Subsidiary\nMalaysia\nEvolutionary Systems Qatar WLL \nStep-down Subsidiary\nQatar\nMastek Systems Pty Ltd (Formerly known as Evolutionary \nSystems Pty Ltd)\nStep-down Subsidiary\nAustralia\nEvolutionary Systems BV \nStep-down Subsidiary\nNetherlands\nEvolutionary Systems (Singapore) Pte. Ltd. \nStep-down Subsidiary\nSingapore\nKey Management Personnel (KMP):\nAshank Desai, Vice Chairman and Managing \nDirector (till March 31,2023)*\nArun Agarwal, Global Chief Financial Officer \n(w.e.f May 31,2021)\nDinesh Kalani, Company Secretary\nGlobal Chief Executive Officer (CEO):\nHiral Chandrana, Global Chief Executive Officer\nDirectors:\nKetan Mehta, Non Executive Director', metadata={'page': 216}),
 Document(page_content='Corporate Information\nBoard of Directors\nAshank Desai\nChairman (Non - Executive) (w.e.f. April 1, 2023) \nVice - Chairman & Managing Director \n(Upto March 21, 2023)\nChairman & Managing Director (Upto March 31, 2023)\nKetan Mehta\nNon - Executive and Non - Independent Director\nRajeev Kumar Grover\nNon - Executive and Independent Director\nSuresh Vaswani\nNon - Executive and Independent Director\n(Appointed w.e.f. December 11, 2022)\nUmang Nahata\nNon-Executive and Non-Independent Director\nNew Shareholders’ Nominee Director\n(Appointed w.e.f. July 19, 2023)\nAtul Kanagat\nNon - Executive and Independent Director\n(Resigned w.e.f. January 17, 2023)\nS. Sandilya\nNon-Executive and Independent Director\n(Resigned w.e.f. March 3, 2023)\nPriti Rao\nNon - Executive and Independent Director', metadata={'page': 333}),
 Document(page_content='engagement through their services.\nAshank Desai\nChairman \n8\nDetails of the highest authority \nresponsible for implementation and \noversight of the Business Responsibility \npolicy (ies)\nThe following people of highest authority shall be responsible for the implementation \nand oversight of the Business Responsibility policy:\nSr. \nNo.\nName of person\nDesignation\nDIN / Employee Id \n1\nMr. Ashank Desai\nChairman \nDIN‑00017767\n2\nMr. Hiral Chandarana\nGroup CEO\nEmployee Id ‑ 83030\n9\nDoes the entity have a specified \nCommittee of the Board/ Director \nresponsible for decision making on \nsustainability related issues? If yes, \nprovide details\nYes; Board, Audit Committee, Nomination and Remuneration Committee, CSR \nCommittee and Risk Management & Governance Committee take decisions related to', metadata={'page': 152}),
 Document(page_content='Non‑Executive Director \n(Independent)\n01.04.2019\nUnion Bank of India – Shareholder \nDirector\n2\n1\nMr. Rajeev Kumar \nGrover (DIN:00058165)\nNon‑Executive Director \n(Independent)\n28.01.2020\n‑\n1\n1\n#Mr. Suresh Vaswani\n(DIN: 02176528)\nNon‑Executive Director \n(Independent)\n11.12.2022\nVodafone Idea Limited – \nIndependent Director\n3\n0\n*Mr. Ashank Desai relinquished the role of Managing Director with effect from March 31, 2023, and has been appointed as Chairman (Non ‑ \nExecutive) with effect from April 1, 2023.\n$Ms. Priti Rao, Non‑Executive & Independent Director submitted resignation from the Directorship of the Company to be effective from May 1, \n2023.\n#Mr. Suresh Vaswani was appointed as an Additional Director of the Company w.e.f. December 11, 2022, which was approved by the Members of the', metadata={'page': 116}),
 Document(page_content='commitments. By embedding ESG considerations into \nour governance framework, we ensure sustainability and \nresponsible business practices are at the forefront of our \ndecision-making processes.\nStrategic Supervision\n• By the Board of Directors \ncomprising the Executive, \nNon-executive and \nIndependent Directors \n• Conducts overall \nstrategic supervision and \ncontrol by setting goals \nand targets, policies, \ngovernance standards, \nreporting mechanism \nand accountability and \ndecision-making process to \nbe followed \nExecutive Management \n• By the corporate \nmanagement team \ncomprising the Managing \nDirector and Executive \nCommittee team consisting \nof the functional heads \nof Mastek\n• Management reviews \nand monitors monthly \nperformances addresses \nchallenges faced by the', metadata={'page': 53}),
 Document(page_content='Management teams promptly for action. The Vice President \n– Group Company Secretary facilitates convening of meetings \nand attends Board, Committee and General Meetings of \nthe Company and maintains the Minutes of these meetings. \nMr. Dinesh Kalani is the Vice President ‑ Group Company \nSecretary of the Company.\nCommittees of the Board – as on March 31, 2023\nThe Board Committees are set up by the Board and \ngoverned by its terms of reference which exhibit the scope, \ncomposition, functioning and reporting parameters. The \nBoard has constituted various Committees to deal with \nspecific business areas. These Committees play an important \nrole in the governance process. All these Committees \nhave been formed with proper Board authority defining', metadata={'page': 120}),
 Document(page_content='For Walker Chandiok & Co LLP\nFor and on behalf of the Board of Directors of Mastek Limited\nChartered Accountants\nFirm Registration No.: 001076N/N500013\nAshank Desai\nRajeev Grover\nChairman\nDirector\nAdi P. Sethna\nDIN: 00017767\nDIN: 00058165\nPartner\nMembership No.: 108840\nArun Agarwal\nDinesh Kalani\nGlobal Chief Financial officer\nVice President - Group Company Secretary\nPlace: Mumbai\nPlace: Mumbai\nDate: April 19, 2023\nDate: April 19, 2023\n181\nShareholder  \nInformation\nStatutory  \nReports\nFinancial\nStatements\nCorporate  \nOverview', metadata={'page': 182}),
 Document(page_content='team comprising of the Managing Director, Chief \nExecutive Officer and Executive Leadership consisting \nof the functional heads of the Company. They meet at \nregular intervals, wherein all important business issues \nare discussed and decisions are taken. Management \nCorporate Governance Report\nCorporate Governance Report\n111\nShareholder  \nInformation\nStatutory  \nReports\nFinancial\nStatements\nCorporate  \nOverview', metadata={'page': 112}),
 Document(page_content='development, corporate governance, finance & taxation, \nsecurity-IT domain expertise and risk management, amongst \nothers. The Company’s Non-Executive Directors (including \nIndependent Directors) have been shaping and steering the \nlong-term strategy and make invaluable contributions towards \nMastek Group level strategy, monitoring or risk management \nand Compliances.\nConsidering the Company’s operations, its expanding \nactivities and valuable contribution made by Non-Executive \nDirectors (including Independent Directors) towards overall \nengagement with the Company on various policies, strategic \nand governance related issues, it is proposed to pay Annual \nCommission to them. In view of the above, the Nomination \nand Remuneration Committee and the Board of Directors', metadata={'page': 312}),
 Document(page_content='As on March 31, 2023, the Board composition and category of the Board of Directors were as follows:\nSr. \nNo.\nName of the Directors\nPromoter / Non–Promoters\nCategory\n1.\n*Mr. Ashank Desai\n Promoter\nChairman & Managing Director\n2.\nMr. Ketan Mehta\nNon‑Executive & Non‑Independent Director\n3.\n$Ms. Priti Rao\n Non – Promoter\nNon‑Executive & Independent Director\n4.\nMr. Rajeev Kumar Grover\n5.\n#Mr. Suresh Vaswani\n*Mr. Ashank Desai relinquished the position of Managing Director with effect from March 31, 2023, and has been appointed as Chairman (Non ‑ \nExecutive) with effect from April 1, 2023.\n$Ms. Priti Rao, Non‑Executive & Independent Director submitted resignation from the Directorship of the Company to be effective from May 1, 2023.', metadata={'page': 114})]